In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

Note: using Google CoLab


In [2]:
# install required packages
!pip install openai
!pip install gradio

In [3]:
# import the packages
import openai
import gradio as gr
import json
from typing import List, Dict, Tuple

client = openai.OpenAI()

# Check if you have set your ChatGPT API successfully
# You should see "Set ChatGPT API sucessfully!!" if nothing goes wrong.
try:
    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages = [{'role':'user','content': "test"}],
            max_tokens=1,
    )
    print("Set ChatGPT API sucessfully!!")
except:
    print("There seems to be something wrong with your ChatGPT API. Please follow our demonstration in the slide to get a correct one.")

Set ChatGPT API sucessfully!!


In [ ]:
# Chatbot

# Initialize an empty list to store messages
messages = []

# Function to reset the conversation
def reset():
    global messages
    messages = []
    return []

# Function to call the model to generate responses
def interact_chatbot(user_input: str, temp=1.0):
    global messages

    # Add user message to messages list
    messages.append({'role': 'user', 'content': user_input})

    # Call the GPT API to generate a response
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temp,
        max_tokens=200,
    )

    # Get the assistant response
    response_text = response.choices[0].message.content

    # Append assistant response to messages list
    messages.append({'role': 'assistant', 'content': response_text})

    # Return the updated chat history
    chat_history = []
    for i in range(0, len(messages), 2):
        if i+1 < len(messages):
            user_msg = messages[i]['content']
            assistant_msg = messages[i+1]['content']
            chat_history.append((user_msg, assistant_msg))

    return chat_history

# Create the Gradio UI interface
with gr.Blocks() as demo:
    gr.Markdown("# Chatbot")
    chatbot = gr.Chatbot()
    user_textbox = gr.Textbox(label="Your Message", interactive=True, placeholder="Type your message here...")
    with gr.Column():
        gr.Markdown("# Temperature\nAdjust the temperature to control the creativity of the responses.")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step=0.1, label="Temperature")
    with gr.Row():
        send_button = gr.Button(value="Send")
        reset_button = gr.Button(value="Reset")

    send_button.click(interact_chatbot, inputs=[user_textbox, temperature_slider], outputs=[chatbot])
    reset_button.click(reset, outputs=[chatbot])

demo.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://9a031991acc702f9f5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
